<a href="https://colab.research.google.com/github/Ki6an/Machine_Learning_Projects/blob/master/Question%20Generation/Question_Generation_1_0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install transformers

!git clone https://github.com/huggingface/transformers

!pip install ./transformers

In [ ]:
from transformers import BertTokenizer, TFBertModel, tf_top_k_top_p_filtering
import tensorflow as tf

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# bert = TFBertModel.from_pretrained("bert-base-uncased") 
tokenizer = BertTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
# bert = 

In [ ]:
from tensorflow.keras import Model
from keras.layers import Dense


class QG_BERT(Model):
    def __init__(self):
        super(QG_BERT, self).__init__()
        self.bert = TFBertModel.from_pretrained("distilbert-base-uncased-distilled-squad")
        self.dense = Dense(tokenizer.vocab_size, activation='softmax')

    def call(self, input):
        embedding = self.bert(input)[0][:, -1, :]
        dense_out = self.dense(embedding)
        # q_string, decoder_out = decoder(input_id, dense_out)
        # input_id = decoder_out
        # print(f'for loop  the input id is {input_id}, and decoder output is {decoder_out} and string is {q_string}')
        return  dense_out

model = QG_BERT()

##Preprocessing the dataset


In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("squad_v2", split='train')

In [ ]:
context = dataset['context'][0:1000]
questions = dataset['question'][0:1000]

In [ ]:
# answers =  [dataset['answers'][i]['text'][0] for i in range(0,1000)]

In [ ]:

# np.savetxt('/content/drive/MyDrive/Machine Learning /Projects/QG/ans_list.txt', answers, delimiter="\n", fmt="%s")

In [ ]:
import numpy as np
lines = np.loadtxt("/content/drive/MyDrive/Machine Learning /Projects/QG/ans_list.txt",dtype=str, delimiter="\n", unpack=True)
answers = lines.tolist()

In [25]:
# features 
inputs = tokenizer(context, answers, padding=True, truncation=True, return_tensors="tf") # gonna take (context, answers) 
# x_ids = inputs['input_ids'] 
# x_mask = inputs['attention_mask'] 
# x_train = [x_ids, x_mask]
inputs =  inputs['input_ids']

# labels
label = tokenizer(questions, padding=True, truncation=True, return_tensors="tf")
label =  label['input_ids']

In [27]:
# # Eg:
# # z = tokenizer.decode(x_train[52]) 
# q = tokenizer.decode(y_train[52])
# z,q
inputs.shape

TensorShape([1000, 434])

In [28]:
def decoder(i, x):
    filtered_next_token_logits = tf_top_k_top_p_filtering(x, top_k=50, top_p=1.0)
    next_token = tf.random.categorical(filtered_next_token_logits, dtype=tf.int32, num_samples=1)
    generated = tf.concat([i, next_token], axis=1)
    output_id = generated.numpy().tolist()[0]
    resulting_string = tokenizer.decode(output_id)
    return resulting_string, generated

In [29]:
def compute_loss(labels, logits):
  loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return loss

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_hat = model(x)
        loss = compute_loss(y, y_hat)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss, y_hat


In [ ]:
##################
# Begin training!#
##################

import numpy 

epochs = 10
from tqdm import tqdm
loss = 1000000.0
history = []


for iter in tqdm(range(epochs)):

    print(f' Epoch {iter} the total loss is {loss}')
    for i, y in enumerate(label):
        print(f' context  {i + 1} loss is {loss}')


        x = inputs[i]
        x = tf.reshape(x,[1,-1])

        
        for word in tqdm(y):
            
            loss, y_hat = train_step(x, word)
            text, x = decoder(x, y_hat)
            history.append(loss.numpy().mean())
            # m = tf.concat(m, 1)

            # plt.plot(history)
            # print(f' for {word} the loss is {loss}')



In [ ]:
import matplotlib.pyplot as plt

plt.plot(history)

In [ ]:
def decoder(i, x):
    filtered_next_token_logits = tf_top_k_top_p_filtering(x, top_k=50, top_p=1.0)
    next_token = tf.random.categorical(filtered_next_token_logits, dtype=tf.int32, num_samples=1)
    generated = tf.concat([i, next_token], axis=1)
    output_id = generated.numpy().tolist()[0]
    resulting_string = tokenizer.decode(output_id)
    return resulting_string, generated

In [ ]:
context = 'my name is kiran'
answers = 'kiran'
inputs = tokenizer.encode(context, answers,padding=True, truncation=True, return_tensors="tf")

In [ ]:
while True: # works as a do-while loop
    dense_out = model(inputs)
    q_string, decoder_out = decoder(inputs, dense_out)
    inputs = decoder_out
    print(f'string is {q_string}')
    if inputs.numpy()[0][-1] == tokenizer.sep_token_id:
        break

In [ ]:
q_string

In [ ]:
# batch_size = 64


# def get_batch(x, batch_size):
#     x = tf.data.Dataset.from_tensor_slices(x)
#     x = x.batch(batch_size)
#     return x

# # x_train = tf.data.Dataset.from_tensor_slices(x_train)
# # x_train = x_train.batch(batch_size)
# # x_train

# x_batch = get_batch(x_train, batch_size)
# y_batch = get_batch(y_train, batch_size)
